In [16]:
pip install crewai 'crewai[tools]' langchain-core langchain-community langchain-google-genai -q

# **Setting Up the Environment and Tools**

In [ ]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import (
    SerperDevTool,
    ScrapeWebsiteTool
)
from crewai.tools import tool

In [3]:
from google.colab import userdata

os.environ['GEMINI_API_KEY'] = userdata.get('gemini_api_key')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [4]:
search_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()

# **Tool Output Check**

In [5]:
search_tool.run(search_query="what is the capital of india")

Using Tool: Search the internet with Serper


{'searchParameters': {'q': 'what is the capital of india',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'organic': [{'title': 'New Delhi - Wikipedia',
   'link': 'https://en.wikipedia.org/wiki/New_Delhi',
   'snippet': 'New Delhi is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government ...',
   'position': 1},
  {'title': 'New Delhi | History, Population, Map, & Facts - Britannica',
   'link': 'https://www.britannica.com/place/New-Delhi',
   'snippet': 'New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of the Yamuna River.',
   'position': 2},
  {'title': 'List of capitals of India - Wikipedia',
   'link': 'https://en.wikipedia.org/wiki/List_of_capitals_of_India',
   'snippet': 'The current capital city is New Delhi, which replaced Calcutta in 1911.',
   'position': 3},
  {'title': 'The Capital of India | Know Your Country | 

# **Configuring the Language Model and Creating Specialized Agents**

In [7]:
def create_llm(api_key=None):
    """Create and configure the language model."""
    return LLM(
        model="gemini/gemini-2.0-flash-lite",
        temperature=0.7,
        api_key=api_key or userdata.get('gemini_api_key')
    )

def create_researcher(topic, llm):
    """Create the researcher agent."""
    return Agent(
        role="Senior Researcher",
        goal=f"Produce comprehensive, balanced, and evidence-based research on {topic}",
        backstory=f"You are an elite research analyst with 15+ years of experience in multidisciplinary research. Your methodology is known for balancing historical context, statistical evidence, diverse expert perspectives, and critical analysis of both challenges and benefits. You have a reputation for uncovering insights others miss and presenting balanced views on complex topics.",
        verbose=True,
        allow_delegation=False,
        tools=[search_tool, scraper_tool],
        llm=llm
    )

def create_writer(topic, llm):
    """Create the writer agent."""
    return Agent(
        role="Senior Content Strategist & Technical Writer",
        goal=f"Create comprehensive, authoritative, and well-structured reports on {topic}",
        backstory=f"You are an award-winning technical writer with expertise in translating complex research into clear, engaging content. You've written for leading publications and research institutions, known for your ability to organize multi-faceted information into logical structures that guide readers through complex topics. Your specialty is creating content that maintains academic rigor while remaining accessible to informed readers.",
        verbose=True,
        llm=llm
    )

# **Defining Research and Writer Tasks**

In [8]:
def create_history_task(topic, researcher):
    """Create task for researching historical events."""
    return Task(
        description=f"Research the history of {topic} and identify the 3 most important events.",
        expected_output="Bullet point list of 3 important events with dates, significance, and impacts.",
        agent=researcher,
        async_execution=True
    )

def create_impacts_task(topic, researcher):
    """Create task for analyzing impacts and controversies."""
    return Task(
        description=f"Research both the controversies/challenges AND the benefits/positive impacts of {topic}.",
        expected_output="Balanced analysis with two sections: (1) 2 key controversies or challenges with different perspectives on each, and (2) 3 significant benefits or positive impacts with supporting evidence.",
        agent=researcher,
        async_execution=True
    )

def create_statistics_task(topic, researcher):
    """Create task for gathering statistical data."""
    return Task(
        description=f"Gather relevant statistical data, studies, and quantitative information about {topic}.",
        expected_output="Organized compilation of 5-10 key statistics with values, units, dates, sources, and brief contextual notes.",
        agent=researcher,
        async_execution=True
    )

def create_insights_task(topic, researcher):
    """Create task for exploring expert insights and future trends."""
    return Task(
        description=f"Research expert insights and future directions for {topic}, ensuring diversity of perspectives.",
        expected_output="Two-part analysis: (1) 3 current key insights from diverse experts (include credentials and backgrounds), and (2) 3 potential future developments or trends according to forward-thinking experts.",
        agent=researcher,
        async_execution=True
    )

def create_writing_task(topic, writer, history_task, impacts_task, insights_task):
    """Create the writing task that synthesizes research findings."""
    return Task(
        description=f"Synthesize all research into a comprehensive, well-structured report on {topic}. The report should integrate historical context, statistical evidence, balanced analysis of impacts (both positive and challenging), expert insights, and future trends.",
        agent=writer,
        expected_output=f"""A detailed, structured report on {topic} with the following sections:

        1. Executive Summary (concise overview of key findings)
        2. Historical Development (key milestones and evolution)
        3. Current Landscape (statistical overview and present state)
        4. Impact Analysis (balanced examination of benefits and challenges)
        5. Expert Perspectives (insights from diverse authorities)
        6. Future Outlook (emerging trends and projected developments)
        7. Conclusion (synthesis of findings and implications)

        Each section should be clearly formatted with headings and subheadings. The report should maintain a balanced perspective throughout and cite information sources where appropriate.""",
        context=[history_task, impacts_task, insights_task],
        output_file="report.md"
    )

# **Assembling the Research Crew**

In [9]:
def create_research_crew(topic, api_key=None):
    """Create and configure the complete research crew."""
    # Initialize core components
    llm = create_llm(api_key)
    researcher = create_researcher(topic, llm)
    writer = create_writer(topic, llm)

    # Create research tasks
    history_task = create_history_task(topic, researcher)
    impacts_task = create_impacts_task(topic, researcher)
    #statistics_task = create_statistics_task(topic, researcher)
    insights_task = create_insights_task(topic, researcher)

    # Create writing task
    writing_task = create_writing_task(
        topic,
        writer,
        history_task,
        impacts_task,
        insights_task
    )

    # Assemble and return the crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[history_task, impacts_task, insights_task, writing_task],
        process=Process.sequential,
        verbose=1
    )

    return crew

In [10]:
def run_research(topic):
    """Execute the complete research workflow."""
    crew = create_research_crew(topic)
    result = crew.kickoff()
    return result

In [15]:
result = run_research("Hayward Fault")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ab280ac8-1dc2-416d-80fb-e826e503f23e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Status: Executing Task...
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Status: Executing Task...
└── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Status: Executing Task...
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Status: Executing Task...
└── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Status: Executing Task...
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Status: Executing Task...
└── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
       Status: Executing Task...

...

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (62)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Search the internet with Serper (57)
│   │   ├── 🔧 Used Search the internet with Serper (59)
│   │   └── 🔧 Used Search the internet with Serper (61)
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   └── 🔧 Used Search the internet with Serper (67)
│   └── 🤖 Agent: Senior Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (71)
│       └── 🔧 Used Read website content (32)
└── 📋 Task: 55527f3f-df57-4478-bfb3-f39d70b28651
       Status: Executing Task...
    └── 🤖 Agent: Senior Content Strategist & Technical Writer
            Status: In Progress

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (62)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Search the internet with Serper (57)
│   │   ├── 🔧 Used Search the internet with Serper (59)
│   │   └── 🔧 Used Search the internet with Serper (61)
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   └── 🔧 Used Search the internet with Serper (67)
│   └── 🤖 Agent: Senior Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (71)
│       └── 🔧 Used Read website content (32)
└── 📋 Task: 55527f3f-df57-4478-bfb3-f39d70b28651
       Status: Executing Task...
    └── 🤖 Agent: Senior Content Strategist & Technical Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5c3ae993-57d5-45d7-ba15-32838ed63560
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: 61a4bc4f-4aa6-4ec9-a5e8-2ff6a9cd939c
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: e0912c91-b302-47eb-b4f7-6c0fc4117cf8
│      Assigned to: Senior Researcher
│      Status: ✅ Completed
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (62)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Search the internet with Serper (55)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Read website content (27)
│   │   ├── 🔧 Used Search the internet with Serper (57)
│   │   ├── 🔧 Used Search the internet with Serper (59)
│   │   └── 🔧 Used Search the internet with Serper (61)
│   ├── 🤖 Agent: Senior Researcher
│   │       Status: ✅ Completed
│   │   └── 🔧 Used Search the internet with Serper (67)
│   └── 🤖 Agent: Senior Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (71)
│       └── 🔧 Used Read website content (32)
└── 📋 Task: 55527f3f-df57-4478-bfb3-f39d70b28651
       Assigned to: Senior Content Strategist & Technical Writer
       Status: ✅ Completed
    └── 🤖 Agent: Senior Content Strategist & Technical Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 55527f3f-df57-4478-bfb3-f39d70b28651                                                                     │
│  Agent: Senior Content Strategist & Technical Writer                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ab280ac8-1dc2-416d-80fb-e826e503f23e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯